<a href="https://colab.research.google.com/github/Bhavyasaradhi/HDS5210_InClass/blob/master/week06/week06_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 6 Exercises

_McKinney 6.1_

There are multiple ways to solve the problems below.  You can use any one of several approaches.  For example, you can read CSV files using Pandas or the csv module.  Your score won't depend on which modules you choose to use unless explicitly noted below, but your programming style will still matter.

### 30.1 List of Allergies

In the /data directory on the Jupyter server, there is a file called `allergies.json` that contains a list of patient allergies.  It is taken from sample data provided by the EHR vendor, Epic, here: https://open.epic.com/Clinical/Allergy

Take some time to look at the structure of the file.  You can open it directly in Jupyter by clicking the _Home_ icon, then the _from_instructor_ folder, and then the _data_ folder.

Within the file, you'll see that it is a dictionary with many items in it.  One of those items is called `entry` and that item is a list of things.  You can tell that because the item name is immediately followed by an opening square bracket, signifying the start of a list.  It's line 11 of the file: `  "entry": [`

Write a function named `allergy_count(json_file)` that takes as one parameter the name of the JSON file and returns an integer number of entries in that file.  Your function should open the file, read the json into a Python object, and return how many items there are in the list of `entry`s.

In [29]:
import json
ALLERGIES_FILE="allergies.json"

In [30]:
import json

def allergy_count(json_file):
    """
    Counts the number of allergy entries in a JSON file.

    Parameters:
    json_file (str): The path to the JSON file containing allergy data.

    Returns:
    int: The number of entries in the 'entry' list of the JSON data.
    """
    with open(json_file) as f:
        data = json.load(f)

    return len(data['entry'])



In [31]:
allergy_count(ALLERGIES_FILE)

4

In [32]:
assert type(allergy_count(ALLERGIES_FILE)) == int
assert allergy_count(ALLERGIES_FILE) == 4

### 30.2 Number of Patients

If you dig a little bit deaper into this list of allergies, you'll see that each result has a patient associated with it.  Create a funcation called `patient_count(json_file)` that will count how many unique patients we have in this JSON structure.  

In [33]:
import json
import numpy as np

def patient_count(ALLERGIES_FILE):
    """
    Counts the number of unique patients in a JSON file containing allergy data.

    Parameters:
    ALLERGIES_FILE (str): The path to the JSON file containing allergy data.

    Returns:
    int: The number of unique patients based on the 'display' name in the JSON data.
    """
    # Load JSON data from the specified file
    data = json.load(open(ALLERGIES_FILE, 'r'))
    temp_list = []

    # Iterate through each entry to extract patient display names
    for patient in data["entry"]:
        if "resource" in patient:
            if "patient" in patient["resource"]:
                temp_list.append(patient["resource"]["patient"]["display"])

    # Using numpy to get a unique list of patient display names
    x = np.array(temp_list)
    uniq_list = np.unique(x)

    return len(uniq_list)




In [34]:
patient_count(ALLERGIES_FILE)

2

In [35]:
assert type(patient_count(ALLERGIES_FILE)) == int
assert patient_count(ALLERGIES_FILE) == 2

### 30.3 How Many Allergies per Patient

Although each entry is a separate allergy, several of them are for the same patient.  Write a function called `allergy_per_patient(json_file)` that counts up how many allergies each patient has.


In [39]:
import json

def allergy_per_patient(ALLERGIES_FILE):
    """
    Counts the number of allergies per patient in a JSON file containing allergy data.

    Parameters:
    ALLERGIES_FILE (str): The path to the JSON file containing allergy data.

    Returns:
    dict: A dictionary where keys are patient display names and values are the count of allergies for each patient.
    """
    # Load JSON data from the specified file
    data = json.load(open(ALLERGIES_FILE, 'r'))
    all_counter = {}  # Dictionary to hold allergy counts per patient

    # Iterate through each entry to extract patient display names and count allergies
    for patient in data["entry"]:
        if "resource" in patient:
            if "patient" in patient["resource"]:
                alg = patient["resource"]["patient"]["display"]
                # Update the allergy count for the patient
                if alg in all_counter:
                    all_counter[alg] += 1
                else:
                    all_counter[alg] = 1

    return all_counter


In [40]:
allergy_per_patient(ALLERGIES_FILE)

{'Jason Argonaut': 3, 'Paul Boal': 1}

In [41]:
assert type(allergy_per_patient(ALLERGIES_FILE)) == dict
assert allergy_per_patient(ALLERGIES_FILE) == {'Paul Boal': 1, 'Jason Argonaut': 3}

### 30.4 Patient Allergies and Reaction

You'll see in the file that each of the items in the `entry` list have several other attributes including a patient name, substance text representation, and a reaction manifestation.  Create a function named `allergy_list(json_file)` that will create an output list that has patient name, allergy, and reaction for each `entry`.  The actual result you should get will be:

```python
[['Jason Argonaut', 'PENICILLIN G', 'Hives'],
 ['Paul Boal', 'PENICILLIN G', 'Bruising'],
 ['Jason Argonaut', 'SHELLFISH-DERIVED PRODUCTS', 'Itching'],
 ['Jason Argonaut', 'STRAWBERRY', 'Anaphylaxis']]
```

You'll notice that the reaction and the manifestation of that action are lists.  You only need to capture the first reaction and the first manifestation of the action.  That is, if there is a list of things, just output the first one.

In [42]:
import json

def allergy_list(ALLERGIES_FILE):
    """
    Compiles a list of allergies for each patient in a JSON file.

    Parameters:
    ALLERGIES_FILE (str): The path to the JSON file containing allergy data.

    Returns:
    list: A list of lists, where each sublist contains:
        - Patient display name
        - Allergy substance
        - Reaction manifestation (if available)
    """
    # Load JSON data from the specified file
    with open(ALLERGIES_FILE, 'r') as f:
        data = json.load(f)

    all_list = []  # List to hold allergy information

    # Iterate through each entry to extract patient and allergy information
    for patient in data.get("entry", []):
        if "resource" in patient:
            resource = patient["resource"]

            name = resource.get("patient", {}).get("display", "Unknown")
            subs = resource.get("substance", {}).get("text", "Unknown")
            mani = "Unknown"  # Default value for manifestation

            if "reaction" in resource:
                react = resource["reaction"]
                if isinstance(react, list) and len(react) > 0:
                    mani = react[0].get("manifestation", [{}])[0].get("text", "Unknown")

            # Append the collected data to the all_list
            all_list.append([name, subs, mani])

    return all_list



In [43]:
allergy_list(ALLERGIES_FILE)

[['Jason Argonaut', 'PENICILLIN G', 'Hives'],
 ['Jason Argonaut', 'SHELLFISH-DERIVED PRODUCTS', 'Itching'],
 ['Jason Argonaut', 'STRAWBERRY', 'Anaphylaxis'],
 ['Paul Boal', 'PENICILLIN G', 'Bruising']]

In [44]:
assert allergy_list(ALLERGIES_FILE) == [['Jason Argonaut', 'PENICILLIN G', 'Hives'],
 ['Jason Argonaut', 'SHELLFISH-DERIVED PRODUCTS', 'Itching'],
 ['Jason Argonaut', 'STRAWBERRY', 'Anaphylaxis'],
 ['Paul Boal', 'PENICILLIN G', 'Bruising']]


### 30.5 Allergy Reaction

Write a function called `allergy_reaction(json_file,patient,substance)` that takes three parameter and returns the reaction that will happen if the patient takes the specified substance.  You can solve this, in part, by calling your `allergy_list` function inside your new `allergy_reaction` function.

If the substance is not found in the allergy list, the function should return None.

In [45]:
import json

def allergy_reaction(json_file, patient, substance):
    """
    Retrieves the reaction associated with a specific patient and substance from a JSON file.

    Parameters:
    json_file (str): The path to the JSON file containing allergy data.
    patient (str): The display name of the patient.
    substance (str): The name of the allergy substance.

    Returns:
    str or None: The reaction manifestation if found, otherwise None.
    """
    # Load JSON data from the specified file
    with open(json_file, 'r') as f:
        data = json.load(f)

    # Search through each entry in the data
    for entry in data.get("entry", []):
        if "resource" in entry:
            resource = entry["resource"]

            # Check if both patient and substance match
            if (resource.get("patient", {}).get("display") == patient and
                resource.get("substance", {}).get("text") == substance):

                # If the substance is found, return its reaction
                if "reaction" in resource and resource["reaction"]:
                    return resource["reaction"][0].get("manifestation", [{}])[0].get("text", "Unknown")

    # If no match is found, return None
    return None



In [46]:
allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'PENICILLIN G')

'Hives'

In [47]:
assert allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'PENICILLIN G') == 'Hives'
assert allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'SHELLFISH-DERIVED PRODUCTS') == 'Itching'
assert allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'STRAWBERRY') == 'Anaphylaxis'
assert allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'PENICILLIN') == None
assert allergy_reaction(ALLERGIES_FILE, 'Paul Boal', 'PENICILLIN G') == 'Bruising'

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

In order to submit your work, you'll need to save this notebook file back to GitHub.  To do that in Google Colab:
1. File -> Save a Copy in GitHub
2. Make sure your HDS5210 repository is selected
3. Make sure the file name includes the week number like this: `week06/week06_assignment_2.ipynb`
4. Add a commit message that means something

**Be sure week names are lowercase and use a two digit week number!!**

**Be sure you use the same file name provided by the instructor!!**

